In [2]:
include("../Polyhedron.jl")
include("interlocking.jl")
include("../plotting.jl")
include("../decomposition.jl")

isconvex

In [ ]:
@time facet_adjacency(Candy)

In [ ]:
reverse!(Candy.facets[1])

In [ ]:
@time adjfacets(Candy, get_facets(Candy)[1])

In [ ]:
@time orient_facets!(Candy)

In [ ]:
@time map(f -> adjfacets(block, f, check = false), get_facets(block))

### Showcase of Interlocking test

In [ ]:
ass, frame = assembly1()

display(plot(ass, width = 360, height = 360))

titest(ass, frame)

### non periodic interlocking

In [ ]:
function read_vertices_faces(filename)
    vertices = Vector{Vector{Float64}}()
    blocks = Vector{Vector{Vector{Int}}}()
    
    open(filename, "r") do file
        last = "0"
        block = Vector{Vector{Int}}()
        for line in eachline(file)
            parts = split(line)
            if isempty(parts)
                continue
            end
            if parts[1] == "v"
                v = parse.(Float64, parts[2:end])
                push!(vertices, v)
                last = "v"
            elseif parts[1] == "f"
                if last == "v" && length(block) > 0
                    push!(blocks, block)
                    block = Vector{Vector{Int}}()
                end
                f = parse.(Int, parts[2:end])
                push!(block, f)
                last = "f"
            end
        end
        push!(blocks, block)
    end
    
    return vertices, blocks
end

function read_frame(filename)
    frame = Vector{Int}()
    open(filename, "r") do file
        for line in eachline(file)
            push!(frame, parse(Int, line))
        end
    end
    return frame
end

In [ ]:
function indexmap(block::Vector{<:Vector{<:Integer}})
    verts = sort(unique(vcat(block...)))
    return i -> findfirst(v -> v == i, verts)
end

function assembly_from_file(ass_file, frame_file)
    v, blocks = read_vertices_faces(ass_file)
    vertices_of_blocks = map(b -> v[sort(unique(vcat(b...)))], blocks)
    facets_of_blocks = map(b -> [indexmap(b).(f) for f in b], blocks)
    ass = [Polyhedron(verts = vertices_of_blocks[i], facets = facets_of_blocks[i]) for i in eachindex(blocks)]
    frame = read_frame(frame_file).+1

    return ass, frame
end

In [ ]:
include("../Polyhedron.jl")

In [ ]:
@time ass_9, frame_9 = assembly_from_file("9.obj", "frame_9.txt")

In [ ]:
block = ass_9[2]

In [ ]:
@time flattenfacets!(block, atol = 1e-8)

In [ ]:
block

In [ ]:
@time triangulate!(block)

In [ ]:
plot(block, labels = true)

In [ ]:
display.(ass_9)

In [ ]:
plot(ass_9[2])

In [ ]:
include("../decomposition.jl")

In [ ]:
@time flattenfacets(ass_9[1])

In [ ]:
titest(ass_9, frame_9)

In [1]:
include("interlocking.jl")
include("../plotting.jl")

┌ Warning: Kaleido is not available on this system. Julia will be unable to save images of any plots.
└ @ PlotlyJS C:\Users\sasch\.julia\packages\PlotlyJS\d9sRW\src\kaleido.jl:65
┌ Warning: ErrorException("Could not start Kaleido process")
└ @ PlotlyJS C:\Users\sasch\.julia\packages\PlotlyJS\d9sRW\src\kaleido.jl:66


plot (generic function with 2 methods)

In [4]:
ass2 = flattenfacets.(assembly);

In [ ]:
include("[-05,05].jl")
ass = [Polyhedron(facets = faces[i], verts = coordinates[i]) for i in eachindex(faces)]
frame = collect(2:9)

framecolor = RGB(1.,0.,0.)
cols = [RGB(0.5,0.5,0.5), RGB(0.,1.,0.), RGB(0.,0.,1.), RGB(0.5,0,0.5), RGB(0.5,0.5,0), RGB(0,0.5,0.5), RGB(0.5, 0.25,0.25), RGB(0.25,0.5,0.25), RGB(0.25,0.25,0.5)]

inds = [1,2,3,4,5,6,7,8,9]
assembly = ass[inds]
fr = Int[2,3,4,5,6,7,8,9]

display(plot(assembly, facetcolors = map(i -> i in fr ? [framecolor] : [cols[i]], eachindex(assembly)), opacity = 0.7))

try
    model = titest(assembly, fr)
    # write(io, model)
catch e
    showerror(e)
end

In [8]:
titest(assembly, fr)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
156 rows, 6 cols, 794 nonzeros
132 rows, 6 cols, 678 nonzeros
Presolve : Reductions: rows 132(-72); columns 6(-48); elements 678(-958)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.2000000000e+04 Ph1: 132(423232); Du: 6(22) 0s
          7     0.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00


┌ Info: The assembly is topologically interlocking as there are no infinitesimal motions of blocks with norm < 1.0e-8. (Maximum norm of infinitesimal motion: 0.0)
└ @ Main c:\Users\sasch\Documents\Code-Projects\GeoCombSurfX\src\Interlocking\interlocking.jl:342


A JuMP Model
Maximization problem with:
Variables: 54
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 156 constraints
`Vector{AffExpr}`-in-`MathOptInterface.Zeros`: 16 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: omega, t